# This script scrapes JavaScript-related vulnerabilities from the IBM X-force Exchange

Import required libraries 

In [1]:
import json
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd

The authentication credentials
Note: The URL query (q) = Javascript

In [7]:
API_KEY = "your_api_key"
API_PASSWORD = "your_api_password"
URL = "https://api.xforce.ibmcloud.com/vulnerabilities/fulltext?q=Javascript"

In [8]:
def get_data_remote(URL):
    js_vulnerabilities_IBM = requests.get(URL, auth=HTTPBasicAuth(API_KEY, API_PASSWORD))
    js_vulnerabilities_IBM_json = json.loads(js_vulnerabilities_IBM.text)
    
    return js_vulnerabilities_IBM_json

In [10]:
js_vulnerabilities_IBM_json

{'total_rows': 1708,
 'bookmark': 'g1AAAAMKeJzLYWBg4MhgTmFQS0lKzi9KdUhJMjTRy0zK1a1Iyy9KTjUwMNRLzskvTUnMK9HLSy3JAapnSmRIkv___38WmJMLJESMDAwtdQ0NdQ1NQgwMrMAoKomBwZsvC81sc0JmJykAySR7bMYb6BoZoRjvsx7deFOCxjuAjI_HYryBpa6RJYrxnn7oxhsSND4BZHw9duONDVCM9_iHbrwRIePzWIAkQwOQAtowH0sAGaAGkFcmuhXGxFmxAGLFfmxWWKBGcRnYClW4FQSjAGLDAYgN97HZYIhqAwO6JwgmUIgVDyBWYEunBqaoVnRkZQEArEHWvw',
 'rows': [{'type': 'vulnerability',
   'xfdbid': 189839,
   'updateid': 109648,
   'inserted': True,
   'variant': 'single',
   'title': 'IBM API Connect cross-site scripting',
   'description': 'IBM API Connect 10.0.0.0 through 10.0.1.0 and 2018.4.1.0 through 2018.4.1.13 is vulnerable to cross-site scripting. This vulnerability allows users to embed arbitrary JavaScript code in the Web UI thus altering the intended functionality potentially leading to credentials disclosure within a trusted session.  IBM X-Force ID:  189839.',
   'risk_level': 5.4,
   'cvss': {'version': '3.0',
    'privilegesrequired': 'Low',
    'userinte

The response is locally saved (just copy and paste into a Notepad and saved it with a `.json` extension) after the first data pull so that I do not need to make the HTTP request in subsequent times as I explore the data.
#### Import the required libraries in the first cell before continuing from here

In [2]:
#Declare the constants - the file paths to loading and saving the data
LOAD_DATA_PATH = r"C:\Users\Semiu\Documents\js-vulnerabilities-dataset\vulData\JavascriptvulFromIBM.json"
SAVE_DATA_PATH = r"C:\Users\Semiu\Documents\js-vulnerabilities-dataset\vulData\extractedvulIBM.csv"

In [3]:
#function to load data from the local machine
def load_data (datapath):
    with open(datapath) as vulData:
        vulData_json = json.load(vulData)
    return vulData_json

In [4]:
#Function to extract data of interest from the loaded data
def extract_data (loaded_data):
    
     #initialize the data dictionary  
    ibm_vuldata = {'title': [], 'description': [], 'exploitability': [], 'risk_level': [], 'cve_id': [], 'platform':[], 'consequences': [], 'privilege':[], 'access_vector':[], 'access_complexity':[], 'confidentiality_impact': [], 'integrity_impact': [], 'availability_impact':[]}
    
    for row in loaded_data['rows']:
        ibm_vuldata['title'].append(row['title'])
        ibm_vuldata['description'].append(row['description'])
        ibm_vuldata['exploitability'].append(row['exploitability'])
        ibm_vuldata['risk_level'].append(row['risk_level'])
        
        #For unexpected missing values observed
        if 'stdcode' in row:
            ibm_vuldata['cve_id'].append(row['stdcode'][0])
        else:
            ibm_vuldata['cve_id'].append(None)
        
        if 'platforms_affected' in row:
            ibm_vuldata['platform'].append(row['platforms_affected'][0])
        else:
            ibm_vuldata['platform'].append(None)
        
        
        ibm_vuldata['consequences'].append(row['consequences'])
        ibm_vuldata['privilege'].append(row['cvss']['privilegesrequired'])
        ibm_vuldata['access_vector'].append(row['cvss']['access_vector'])
        ibm_vuldata['access_complexity'].append(row['cvss']['access_complexity'])
        ibm_vuldata['confidentiality_impact'].append(row['cvss']['confidentiality_impact'])
        ibm_vuldata['integrity_impact'].append(row['cvss']['integrity_impact'])
        ibm_vuldata['availability_impact'].append(row['cvss']['availability_impact'])
            
    return ibm_vuldata

In [5]:
#function to save the data in csv to local machine
def save_data_tocsv (ibm_vuldata):
    
    IBM_vuldata_frame = pd.DataFrame(ibm_vuldata)
    IBM_vuldata_frame.to_csv(SAVE_DATA_PATH, encoding='utf-8')

In [6]:
#Calling the three functions in a statement - functional programming aye
#when data is loaded from the local
save_data_tocsv(extract_data (load_data(LOAD_DATA_PATH)))

In [8]:
#check
loaded_data = load_data(LOAD_DATA_PATH)


In [13]:
len(loaded_data['rows'])

200

In [ ]:
#when data is loaded from the remote
save_data_tocsv(extract_data (get_data_remote(URL)))